In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files
import io

In [ ]:
# File upload
print("\n Please upload your file")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
orders = pd.read_csv(io.BytesIO(uploaded[filename]), sep=',')
print(f"\nShape: {orders.shape}\nFirst 5 rows:")
print(orders.head())


 Please upload your file


Saving sample-data.csv to sample-data.csv

Shape: (25613, 4)
First 5 rows:
  order_date        order_id          customer  grand_total
0   9/7/2011  CA-2011-100006       Dennis Kane        378.0
1   7/8/2011  CA-2011-100090        Ed Braxton        699.0
2  3/14/2011  CA-2011-100293  Neil Franz?sisch         91.0
3  1/29/2011  CA-2011-100328   Jasper Cacioppo          4.0
4   4/8/2011  CA-2011-100363       Jim Mitchum         21.0


In [ ]:
#  Data preparation
print("\n Converting order_date to datetime...")
orders['order_date'] = pd.to_datetime(orders['order_date'])
testeddate = '2014/12/31'
NOW = datetime.strptime(testeddate, '%Y/%m/%d')


 Converting order_date to datetime...


In [ ]:
# RFM Table calculation
print("\n Calculating RFM Table...")
rfmTable = orders.groupby('customer').agg({
    'order_date': lambda x: (NOW - x.max()).days, # Recency, as days
    'order_id': 'count',                         # Frequency
    'grand_total': 'sum'                         # Monetary
})

rfmTable.rename(columns={
    'order_date': 'recency',
    'order_id': 'frequency',
    'grand_total': 'monetary_value'
}, inplace=True)
print(rfmTable.head())


 Calculating RFM Table...
                 recency  frequency  monetary_value
customer                                           
Aaron Bergman        415          3           887.0
Aaron Hawkins         12          7          1744.0
Aaron Smayling        88          7          3050.0
Adam Bellavance       54          8          7756.0
Adam Hart             34         10          3249.0


In [ ]:
# Step 5: Quantiles computation
quantiles = rfmTable.quantile(q=[0.25, 0.5, 0.75]).to_dict()
print("\nQuantiles summary:")
print(quantiles)


Quantiles summary:
{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0}, 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0}, 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}


In [ ]:
# RFM Score functions
def RClass(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

def FMClass(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

print("\n🔢 Calculating quartiles and segment scores...")
rfmTable['R_Quartile'] = rfmTable['recency'].apply(RClass, args=('recency', quantiles,))
rfmTable['F_Quartile'] = rfmTable['frequency'].apply(FMClass, args=('frequency', quantiles,))
rfmTable['M_Quartile'] = rfmTable['monetary_value'].apply(FMClass, args=('monetary_value', quantiles,))
rfmTable['RFMClass'] = rfmTable['R_Quartile'].astype(str) + rfmTable['F_Quartile'].astype(str) + rfmTable['M_Quartile'].astype(str)
print(rfmTable.head())


🔢 Calculating quartiles and segment scores...
                 recency  frequency  monetary_value  R_Quartile  F_Quartile  \
customer                                                                      
Aaron Bergman        415          3           887.0           4           4   
Aaron Hawkins         12          7          1744.0           1           2   
Aaron Smayling        88          7          3050.0           3           2   
Adam Bellavance       54          8          7756.0           2           2   
Adam Hart             34         10          3249.0           2           1   

                 M_Quartile RFMClass  
customer                              
Aaron Bergman             4      444  
Aaron Hawkins             3      123  
Aaron Smayling            2      322  
Adam Bellavance           1      221  
Adam Hart                 2      212  


In [ ]:
# Top customers by RFM
print("\nTop 'Champions' (RFMClass 111):")
champions = rfmTable[rfmTable['RFMClass'] == '111'].sort_values('monetary_value', ascending=False).head(5)
print(champions)


Top 'Champions' (RFMClass 111):
                recency  frequency  monetary_value  R_Quartile  F_Quartile  \
customer                                                                     
Sanjit Engle          9         11         12210.0           1           1   
John Lee             21         11          9801.0           1           1   
Pete Kriz             9         12          8647.0           1           1   
Harry Marie           2         10          8237.0           1           1   
Lena Creighton       16         12          7661.0           1           1   

                M_Quartile RFMClass  
customer                             
Sanjit Engle             1      111  
John Lee                 1      111  
Pete Kriz                1      111  
Harry Marie              1      111  
Lena Creighton           1      111  


In [ ]:
# Step 8: Total Score and Label
rfmTable['Total Score'] = rfmTable['R_Quartile'] + rfmTable['F_Quartile'] + rfmTable['M_Quartile']

def assign_label(score):
    if score == 12:
        return "Excellent"
    elif score >= 7:
        return "Good"
    elif score >= 3:
        return "Bad"
    else:
        return "Only 1 transaction?"

print("\n🏷️ Assigning labels...")
rfmTable['Label'] = rfmTable['Total Score'].apply(assign_label)
print(rfmTable['Label'].value_counts())


🏷️ Assigning labels...
Label
Good         437
Bad          289
Excellent     67
Name: count, dtype: int64


In [ ]:
# Save results and visualization
print("\nSaving RFM segmentation to CSV: 'rfm-table-data.csv'")
rfmTable.to_csv('rfm-table-data.csv', sep=',')

def color(val):
    if val == "Excellent":
        return 'background-color: green'
    elif val == "Good":
        return 'background-color: yellow'
    elif val == "Bad":
        return 'background-color: red'
    return 'background-color: white'

print("\nYou can view a styled summary in a dataframe below (not downloadable):")
rfmTable.style.applymap(color, subset=['Label'])


Saving RFM segmentation to CSV: 'rfm-table-data.csv'

You can view a styled summary in a dataframe below (not downloadable):


/tmp/ipython-input-622009190.py:15: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  rfmTable.style.applymap(color, subset=['Label'])


,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Total Score,Label
customer,,,,,,,,,
Aaron Bergman,415,3,887.000000,4,4,4,444,12,Excellent
Aaron Hawkins,12,7,1744.000000,1,2,3,123,6,Bad
Aaron Smayling,88,7,3050.000000,3,2,2,322,7,Good
Adam Bellavance,54,8,7756.000000,2,2,1,221,5,Bad
Adam Hart,34,10,3249.000000,2,1,2,212,5,Bad
Adam Shillingsburg,28,9,3256.000000,1,1,2,112,4,Bad
Adrian Barton,41,10,14474.000000,2,1,1,211,4,Bad
Adrian Hane,60,7,1734.000000,2,2,3,223,7,Good
Adrian Shami,41,2,59.000000,2,4,4,244,10,Good
